In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import math

In [2]:
export_dir = './final_model_9/'
batch_size = 10000
disp_log = 50
round_to_int = True

In [3]:
def getAreaSeriesFromIdx(s):
    #f = s.mask(s['in_area'], s['lon_g']*3+s['lat_g']+1, axis=0)['lat']
    f = s.mask(s['in_area'], s['lat_g']+1, axis=0)['lat']
    f = f.where(s['in_area'], 0)
    return f

In [4]:
area_num = 4
dw = pd.read_csv('./DL_final_project/DL_Taiwan_data/sinica/201703_Taiwan.csv')
dw = dw.assign(Timestamp = pd.to_datetime(dw['Date']+' '+dw['Time']))
dw = dw.assign(Hour = lambda x: x['Timestamp'].dt.hour)
dw = dw.assign(in_area = lambda x:(x['lat']>= 21 )& (x['lat'] <= 27)&(x['lon']>=118)&(x['lon']<=122))
dw = dw.assign(lon_g = lambda x:(x.lon-118)//2)
dw = dw.assign(lat_g = lambda x:(x.lat-21)//2)
dw = dw.assign(area = getAreaSeriesFromIdx)
dw = dw.assign(idx = dw.index)
n_batches = math.ceil(len(dw) / batch_size)
print('Total batches:',n_batches)

Total batches: 420


In [5]:
print(len(dw))

4191423


In [6]:
result_pd = pd.DataFrame({'PM2.5':[],'idx':[]})
for a in range(area_num):
#if True:
    #a = 0
    print('Area', a)
    tf.reset_default_graph()
    dft = dw[dw['area']==a]
    predict_values = np.zeros(len(dft))
    print(len(dft))
    df_X = dft[['Hour','PM10','PM1','Temperature','Humidity','lon','lat']]
    df_X = df_X.values
    
    n_batches = math.ceil(len(dft) / batch_size)
    with tf.Session(graph=tf.Graph()) as sess:
        tf.saved_model.loader.load(sess, ["tag"], export_dir + 'area_'+str(a)+"/")
        graph = tf.get_default_graph()
        X = graph.get_tensor_by_name("Input/X:0")
        is_training = graph.get_tensor_by_name("Input/is_training:0")
        predict = graph.get_tensor_by_name("Predict/predict:0")

        for i in range(n_batches):
            if i % disp_log == 0:
                print('batch',i)
            if i == n_batches - 1:
                X_batch = df_X[i*batch_size:]
                predict_batch = sess.run(predict, {is_training: False, X: X_batch})
                predict_values[i*batch_size:] = predict_batch[:,0]
            else:
                X_batch = df_X[i*batch_size:(i+1)*batch_size]
                predict_batch = sess.run(predict, {is_training: False, X: X_batch})
                predict_values[i*batch_size:(i+1)*batch_size] = predict_batch[:,0]


        if round_to_int:
            predict_values = [int(v) for v in predict_values]
    
    area_result = pd.DataFrame({'PM2.5':predict_values, 'idx':dft['idx']})
    result_pd = result_pd.append(area_result)
print('end')

Area 0
359855
INFO:tensorflow:Restoring parameters from b'./final_model_9/area_0/variables\\variables'
batch 0
Area 1
858290
INFO:tensorflow:Restoring parameters from b'./final_model_9/area_1/variables\\variables'
batch 0
batch 50
Area 2
2073941
INFO:tensorflow:Restoring parameters from b'./final_model_9/area_2/variables\\variables'
batch 0
batch 50
batch 100
batch 150
batch 200
Area 3
899337
INFO:tensorflow:Restoring parameters from b'./final_model_9/area_3/variables\\variables'
batch 0
batch 50
end


In [7]:
result = result_pd.sort_values('idx')
result

,PM2.5,idx
0,39.0,0.0
1,49.0,1.0
2,50.0,2.0
3,36.0,3.0
4,43.0,4.0
5,77.0,5.0
6,42.0,6.0
7,55.0,7.0
8,49.0,8.0
9,23.0,9.0


In [8]:
result.drop(['idx'], axis=1).to_csv('predict_upload_multidnn.csv', float_format='%.f', index=False)
print('finish')

finish
